In [95]:
import numpy as np
import pandas as pd

import pickle
import ast
from itertools import combinations

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn import set_config
from sklearn.utils import shuffle
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score ,train_test_split, LeaveOneOut
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, mutual_info_regression

import category_encoders as ce
import missingno as msno

from patsy import dmatrices
from statsmodels.compat import lzip
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.compat import lzip

from constants import *

%matplotlib inline

In [96]:
set_config(transform_output="pandas")

In [97]:
html_folder = 'C:\\Users\\Brayden\\Desktop\\Personal Website\\Brayden-L.github.io\\_includes\\complex_linear_regression_routes\\'

In [98]:
df = pd.read_pickle('All_Loc_Cleaned_Stripped.pkl')

In [99]:
# This undoes the complex length imputation performed in "Length_Imputation.ipynb" for nullity analysis

undo_length_imp_bool = False

if undo_length_imp_bool:
        df1 = pd.read_csv('All_Loc.csv')
        # Create Route ID Column for joining and remove duplicates
        if "Route ID" not in df1.columns:
                df1.insert(len(df1.columns), "Route ID", "")
        df1["Route ID"] = df1["URL"].apply(lambda x: int(x.split("/")[4]))

        df1.drop_duplicates('Route ID', inplace=True)
        df1[['Route ID', 'Length']]
        df.drop('Length', axis=1, inplace=True)
        df = df.merge(df1[['Route ID', 'Length']], how='left', on='Route ID')

In [100]:
df = shuffle(df)

In [101]:
star_vote_cutoff = 5
bayesian_stars=True


# Cleaning

In [102]:
# Route ID
df.set_index('Route ID', inplace=True)

## Response Variable Cleaning

In [103]:
o_size = df.shape[0]

In [104]:
# Drop values with not enough vote ratings
df.drop(df[df['Num Star Ratings']<=star_vote_cutoff].index, axis=0, inplace=True)

In [105]:
# Drop MP entries
df.drop(df[df['SP/MP']=='MP'].index, axis=0, inplace=True)

In [106]:
df.shape[0]

12990

In [107]:
df.shape[0]/o_size

0.449620989235402

In [108]:
if bayesian_stars:
    m = df['Avg Stars'].mean()
    c = df['Num Star Ratings'].quantile(0.25)
    df['Bayesian Stars'] = ((df['Avg Stars'] * df['Num Star Ratings']) + (c*m)) / (df['Num Star Ratings'] + c)

    # Compare old rating scale to new
    x0 = df['Avg Stars']
    x1 = df['Bayesian Stars']
    rate_comp_df =pd.DataFrame(dict(
        series=np.concatenate((["Average Stars"]*len(x0), ["Bayesian Stars"]*len(x1))), 
        data  =np.concatenate((x0,x1))
    ))
    fig = px.histogram(rate_comp_df, color='series', barmode='overlay', marginal='box')
    fig.update_layout(title='Avg Vs. Bayesian Stars Histogram', title_x=0.5)
    fig.update_xaxes(title="Stars", row=1,col=1)
    fig.update_yaxes(title="Count")
    # fig.write_html(html_folder + 'Avg_Vs_Baye_Hist.html')
    fig.show()
else:
    fig = px.histogram(df['Avg Stars'])
    fig.show()

In [109]:
# Homogenize Rating
def grade_homo(df_source, r_type, r_direction, b_type, b_direction):
    """
    Reassigns grades to a single YDS or Vgrade schema.

    Parameters
    ----------
    df_source : df
        Original route df.
    r_type : str [letter, sign]
        YDS letter or sign style grades.
    r_direction : str [up, down, even_rand, manual]
        Unused if r_type='letter'. Which way to assign grades. even_rand rounds a randomly selected half up and the randomly remaining half down.
    b_type : str [flat, sign]
        Vgrade flat grades or include sign grades.
    b_direction : str [up, down, even_rand, manual]
        Used for both b_type.

    Return
    ------
    df_source : df
        Original df with grade homogenization
    """
    rating_isolate = df_source["Original Rating"].apply(
        lambda row: [val for val in row.split()][0]
    )  # This is a fail-safe to ensure we are only looking at the part of the rating we care about, not risk or sub-ratings.

    # Reset 'Rating' column so this mapping can be re-run
    df_source["Rating"] = df_source["Original Rating"]

    # Roped Grades
    def grademoderate():
        grade_change_subset = rating_isolate.isin(list(rgrademoderatemap.keys()))
        df_source.loc[grade_change_subset, "Rating"] = df_source.loc[
            grade_change_subset
        ]["Original Rating"].map(rgrademoderatemap)

    def grade_split(upmap, downmap):
        grade_change_subset = rating_isolate.isin(list(upmap.keys()))
        grade_change_subset_df = df_source[grade_change_subset]
        for grade in grade_change_subset_df["Original Rating"].unique():
            to_change = grade_change_subset_df[
                grade_change_subset_df["Original Rating"] == grade
            ]
            changed_up = to_change.sample(frac=0.5)["Original Rating"].map(upmap)
            df_source.loc[changed_up.index, "Rating"] = changed_up
        grade_change_subset = rating_isolate.isin(list(downmap.keys()))
        grade_change_subset_df = df_source[grade_change_subset]
        for grade in grade_change_subset_df["Original Rating"].unique():
            to_change = grade_change_subset_df[
                grade_change_subset_df["Original Rating"] == grade
            ]
            changed_down = to_change["Original Rating"].map(downmap)
            df_source.loc[changed_down.index, "Rating"] = changed_down

    if r_type == "sign":
        grade_change_subset = rating_isolate.isin(list(rgradecompmap.keys()))
        df_source.loc[grade_change_subset, "Rating"] = df_source[grade_change_subset][
            "Original Rating"
        ].map(rgradecompmap)
    else:
        if r_direction == "up":
            grademoderate()
            grade_change_subset = rating_isolate.isin(list(rgradeupmap.keys()))
            df_source.loc[grade_change_subset, "Rating"] = df_source[
                grade_change_subset
            ]["Original Rating"].map(rgradeupmap)
        if r_direction == "down":
            grademoderate()
            grade_change_subset = rating_isolate.isin(list(rgradedownmap.keys()))
            df_source.loc[grade_change_subset, "Rating"] = df_source[
                grade_change_subset
            ]["Original Rating"].map(rgradedownmap)
        if r_direction == "even_rand":
            grademoderate()
            grade_split(rgradeupmap, rgradedownmap)

    # Boulder Grades
    if b_type == "flat":
        # Remove all + and - grades
        grade_change_subset = rating_isolate.isin(list(bgradeconmapflat.keys()))
        df_source.loc[grade_change_subset, "Rating"] = df_source[grade_change_subset][
            "Original Rating"
        ].map(bgradeconmapflat)

        if b_direction == "up":
            grade_change_subset = rating_isolate.isin(list(bgradeupmapflat.keys()))
            df_source.loc[grade_change_subset, "Rating"] = df_source[
                grade_change_subset
            ]["Original Rating"].map(bgradeupmapflat)
        if b_direction == "down":
            grade_change_subset = rating_isolate.isin(list(bgradedownmapflat.keys()))
            df_source.loc[grade_change_subset, "Rating"] = df_source[
                grade_change_subset
            ]["Original Rating"].map(bgradedownmapflat)
        if b_direction == "even_rand":
            grade_split(bgradeupmapflat, bgradedownmapflat)

    if b_type == "sign":
        if b_direction == "up":
            grade_change_subset = rating_isolate.isin(list(bgradeupmapsign.keys()))
            df_source.loc[grade_change_subset, "Rating"] = df_source[
                grade_change_subset
            ]["Original Rating"].map(bgradeupmapsign)
        if b_direction == "down":
            grade_change_subset = rating_isolate.isin(list(bgradedownmapsign.keys()))
            df_source.loc[grade_change_subset, "Rating"] = df_source[
                grade_change_subset
            ]["Original Rating"].map(bgradedownmapsign)
        if b_direction == "even_rand":
            grade_split(bgradeupmapsign, bgradedownmapsign)

    return df_source

df['Original Rating'] = df['Rating']
df = grade_homo(df, 'letter', 'even_rand', 'flat', 'down')
df.drop('Original Rating', axis=1, inplace=True)

## Predictor Cleaning

### Null Handling

In [110]:
null_plot_bool = False

nulldf = df[['Rating', 'Length', 'Area Latitude', 'Area Longitude', 'Risk', 'Num Ticks', 'Lead Ratio', 'OS Ratio', 'Repeat Sender Ratio', 'Mean Attempts To RP', 'Route Type']]

In [111]:
if null_plot_bool:
    msno.matrix(nulldf)

In [112]:
if null_plot_bool:
    msno.heatmap(nulldf)

In [113]:
if null_plot_bool:
    msno.dendrogram(nulldf)

In [114]:
(nulldf.isnull().sum()/df.shape[0])*100

Rating                  0.246343
Length                  0.000000
Area Latitude           0.000000
Area Longitude          0.000000
Risk                   93.941493
Num Ticks               0.046189
Lead Ratio              0.400308
OS Ratio                2.609700
Repeat Sender Ratio     3.903002
Mean Attempts To RP    28.221709
Route Type              0.007698
dtype: float64

In [115]:
# Remove null rating and route type values which are likely 3rd, 4th class etc. Not relevant.
df.drop(df[df['Rating'].isna()].index, inplace=True)
df.drop(df[df['SP/MP'].isna()].index, inplace=True)
df.drop(df[df['Route Type'].isna()].index, inplace=True)
df.drop(df[df['Route Type']=='Boulder'].index, inplace=True)

In [116]:
df.shape[0]

12946

### Encoding

In [117]:
# Risk
# Replace NA with 'G'
df['Risk'] = df['Risk'].cat.set_categories(['G', 'PG13', 'R', 'X'], ordered=True)
df.loc[df['Risk'].isna(), 'Risk'] = 'G'

In [118]:
# Change Trad/Sport column to "Is Trad"
df['Is Trad'] = df['Route Type'].map({"Trad":1, "Sport":0}).astype(int)
df.drop('Route Type', axis=1, inplace=True)
# Change SP/MP solumn to "Is SP"
df['Is SP'] = df['SP/MP'].map({'SP':1, 'MP':0}).astype(int)
df.drop('SP/MP', axis=1, inplace=True)
# Change Length Missing to binary
df['Length Missing'] = df['Length Missing'].map({True:1, False:0}).astype(int)

In [119]:
# Encode Risk Ordinal
ce_ord_risk = ce.OrdinalEncoder(cols=['Risk'])
df = ce_ord_risk.fit_transform(df)

In [120]:
# Encode Rating Ordinal
ce_ord_rating = ce.OrdinalEncoder(cols=['Rating'])
df = ce_ord_rating.fit_transform(df)

In [121]:
# sns.pairplot(df[['Avg Stars', 'Lead Ratio', 'OS Ratio', 'Repeat Sender Ratio', 'Mean Attempts To RP']])

### Imputation

In [122]:
# Length imputation was done prior to this on the whole data set. It is a pretty involved custom imputation.

In [123]:
# Num Ticks and Num Tickers
df.loc[df['Num Ticks'].isna(), 'Num Ticks'] = 0
df.loc[df['Num Tickers'].isna(), 'Num Tickers'] = 0

In [169]:
preproc_feature_list = ['Rating', 'Length', 'Area Latitude', 'Area Longitude', 'Risk', 'Num Ticks', 'Lead Ratio', 'OS Ratio', 'Repeat Sender Ratio', 'Mean Attempts To RP', 'Is Trad']

tick_metric_simp_imp_bool = True
missing_ind_bool = False

if tick_metric_simp_imp_bool:
    feat_impute_method = make_column_transformer((SimpleImputer(strategy='median', add_indicator=missing_ind_bool), ['OS Ratio', 'Lead Ratio']), 
                                                 (SimpleImputer(strategy='constant', fill_value=1, add_indicator=missing_ind_bool), ['Repeat Sender Ratio', 'Mean Attempts To RP']), 
                                                 remainder='passthrough', verbose_feature_names_out=False)
else:
    feat_impute_method = make_column_transformer((IterativeImputer(min_value=0, add_indicator=missing_ind_bool), preproc_feature_list), 
                                                 remainder='passthrough', verbose_feature_names_out=False)
feat_impute_method.fit_transform(df[preproc_feature_list])

,OS Ratio,Lead Ratio,Repeat Sender Ratio,Mean Attempts To RP,Rating,Length,Area Latitude,Area Longitude,Risk,Num Ticks,Is Trad
Route ID,,,,,,,,,,,
106254851,0.500000,0.778443,1.144828,1.416667,1,52.0,43.80273,-71.83029,1,458.0,0
107246115,0.846591,0.716724,1.025974,1.375000,2,90.0,49.66932,-123.15753,1,424.0,1
117126111,0.634615,0.940678,1.000000,1.153846,3,60.0,44.13663,-107.25768,1,136.0,0
105993590,0.912281,0.896552,1.076923,1.000000,4,30.0,43.80313,-71.83493,1,152.0,0
105943599,0.741935,0.395062,1.037037,1.000000,5,80.0,38.08299,-81.08043,1,116.0,1
...,...,...,...,...,...,...,...,...,...,...,...
105801735,0.619048,0.758065,1.000000,1.250000,1,185.0,44.36716,-121.14353,1,73.0,0
112007797,0.807692,0.735632,1.068182,1.000000,7,100.0,39.44184,-120.65722,1,102.0,0
106003617,0.050000,0.893617,1.000000,1.038462,18,65.0,44.13246,-107.25716,1,67.0,0


In [125]:
(df.isnull().sum()/df.shape[0])*100

Route                   0.000000
Location                0.000000
Avg Stars               0.000000
Num Star Ratings        0.000000
Rating                  0.000000
Pitches                 0.000000
Length                  0.000000
Length Missing          0.000000
Area Latitude           0.000000
Area Longitude          0.000000
Risk                    0.000000
Base Location           0.000000
Num Ticks               0.000000
Num Tickers             0.000000
Lead Ratio              0.347598
OS Ratio                2.549050
Repeat Sender Ratio     3.823575
Mean Attempts To RP    28.070446
Bayesian Stars          0.000000
Is Trad                 0.000000
Is SP                   0.000000
dtype: float64

## Outliers

In [126]:
# Remove outliers
repeat_sender_ratio_cutoff = 1.6
mean_attempt_to_rp_cutoff = 3.1
print(df[df['Repeat Sender Ratio']>repeat_sender_ratio_cutoff].shape[0])
df.drop(df[df['Repeat Sender Ratio']>repeat_sender_ratio_cutoff].index, axis=0, inplace=True)
print(df[df['Mean Attempts To RP']>mean_attempt_to_rp_cutoff].shape[0])
df.drop(df[df['Mean Attempts To RP']>mean_attempt_to_rp_cutoff].index, axis=0, inplace=True)

93
59


In [127]:
# 28% of values are null
# 29% are a near default 1
# 2.1% are outliers

In [128]:
# px.histogram(df['Mean Attempts To RP'], marginal='box')

In [129]:
# px.histogram(df[df['Mean Attempts To RP']>1]['Mean Attempts To RP'], marginal='box')

In [130]:
# 3.8% are null
# 58.5% are near default 1
# 3.4% outliers

In [131]:
# px.histogram(df['Repeat Sender Ratio'], marginal='box')

In [132]:
# px.histogram(df[df['Repeat Sender Ratio']>1]['Repeat Sender Ratio'], marginal='box')

In [133]:
df.shape[0]

12794

## EDA

In [134]:
# sns.pairplot(df[['Rating', 'Is Trad', 'Risk', 'Length', 'Area Latitude', 'Area Longitude', 'Num Ticks', 'Lead Ratio', 'OS Ratio', 'Repeat Sender Ratio', 'Mean Attempts To RP']], kind='kde', diag_kind='kde', corner=True)

In [135]:
# sns.pairplot(df[['Rating', 'Is Trad', 'Risk', 'Length', 'Area Latitude', 'Area Longitude', 'Num Ticks', 'Lead Ratio', 'OS Ratio', 'Repeat Sender Ratio', 'Mean Attempts To RP']], kind='reg', diag_kind='kde', corner=True, plot_kws={'line_kws':{'color':'red'}})

In [136]:
### Correlation Heatmap
# Calculate correlation using the default method ( "pearson")
corr = df[['Rating', 'Is Trad', 'Risk', 'Length', 'Area Latitude', 'Area Longitude', 'Num Ticks', 'Lead Ratio', 'OS Ratio', 'Repeat Sender Ratio', 'Mean Attempts To RP']].corr()
# optimize aesthetics: generate mask for removing duplicate / unnecessary info
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
# Generate a custom diverging colormap as indicator for correlations:
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Plot
sns.set(rc={"figure.figsize":(12, 12)})
# sns.heatmap(corr, mask=mask, cmap=cmap, annot=True,  square=True, annot_kws={"size": 12}, vmin=-1, vmax=1)

In [137]:
df.drop(['Location', 'Base Location', 'Num Tickers', 'Pitches', 'Is SP', 'Route'], axis=1, inplace=True)

## Scale

In [138]:
std_scaler = StandardScaler()
normalize_scaler = MinMaxScaler(feature_range = (0, 1))
robust_scaler = RobustScaler()
quant_scaler = QuantileTransformer()
power_scaler = PowerTransformer()

scaler_list = (std_scaler, normalize_scaler, robust_scaler, quant_scaler, power_scaler)
scaler_sel = make_column_transformer((quant_scaler, preproc_feature_list), remainder='passthrough', verbose_feature_names_out=False)

## Preproc Pipeline

In [139]:
# Define preproc pipeline
preproc_pipe = Pipeline([
    ("imputer", feat_impute_method),
    ("scaler", scaler_sel)
])

# Pipeline Setup

In [140]:
# Create hold-out test set
feature_col_ful = df[['Rating', 'Is Trad', 'Risk', 'Length', 'Area Latitude', 'Area Longitude', 'Num Ticks', 'Lead Ratio', 'OS Ratio', 'Repeat Sender Ratio', 'Mean Attempts To RP']]
X = feature_col_ful
y = df['Bayesian Stars']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

## Basic Linear Regression

In [141]:
basic_linreg = LinearRegression()
basic_linreg_pipe = Pipeline([('preproc', preproc_pipe), ('model', basic_linreg)])
print(np.mean(np.sqrt(-cross_val_score(basic_linreg_pipe, X_train, y_train, cv=5, scoring='neg_mean_squared_error'))))
print(np.mean(cross_val_score(basic_linreg_pipe, X_train, y_train, cv=5)))

0.36731504361700945
0.381884439173006


In [3715]:
# SequentialFeatureSelector isn't set up to work with pipelines nicely, so we have to do it semi-manually.
X_train_forward_subselect = preproc_pipe.fit_transform(X_train)
sfs = SequentialFeatureSelector(basic_linreg, n_features_to_select='auto', tol=0.001, cv=5, scoring='neg_mean_squared_error')
sfs.fit(X_train_forward_subselect, y_train)
sfs.get_support
sfs.transform(X_train_forward_subselect)

,Is Trad,Length,Num Ticks,Lead Ratio,OS Ratio
0,0.0,0.480480,0.437437,0.636637,0.826326
1,0.0,0.374875,0.790791,0.472815,0.508366
2,0.0,0.570070,0.421421,0.865365,0.098829
3,1.0,0.374875,0.634134,0.150216,0.826326
4,1.0,0.706707,0.485986,0.052252,1.000000
...,...,...,...,...,...
11509,0.0,0.570070,0.164665,0.853353,0.000000
11510,0.0,0.783784,0.024525,1.000000,0.000000
11511,0.0,0.570070,0.903904,0.809794,0.397248
11512,0.0,0.706707,0.777778,0.582119,0.570571


In [171]:
X_train_mi = preproc_pipe.fit_transform(X_train)

mi_dat = mutual_info_regression(X_train_mi, y_train)
print(pd.DataFrame(data=mi_dat, index=X_train_mi.columns, columns=['MI']).sort_values('MI', ascending=False))

sfs = SelectKBest(mutual_info_regression, k=5)
sfs.fit_transform(X_train_mi, y_train)
sfs.get_support
print(sfs.transform(X_train_mi).columns)

                           MI
Num Ticks            0.543946
OS Ratio             0.265808
Mean Attempts To RP  0.224947
Rating               0.213521
Repeat Sender Ratio  0.171278
Lead Ratio           0.124675
Area Latitude        0.116688
Area Longitude       0.115700
Length               0.086297
Is Trad              0.014288
Risk                 0.005220
Index(['Rating', 'Num Ticks', 'OS Ratio', 'Repeat Sender Ratio',
       'Mean Attempts To RP'],
      dtype='object')


In [173]:
pls_model = Ridge()
pls_mse=[]

for i in np.arange(1, X_train.shape[1]+1):
    pls_model = PLSRegression(n_components=i)
    pls_pipe = Pipeline([('preproc', preproc_pipe), ('pls_model', pls_model)])
    score = cross_val_score(pls_pipe, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()
    pls_mse.append(-score)
    
fig = px.scatter(pd.DataFrame(np.sqrt(pls_mse), np.arange(1, X_train.shape[1]+1)))
fig.update_xaxes(title='Number of Features')
fig.update_yaxes(title='RMSE')
fig.update_layout(showlegend=False, title='RMSE Vs. Num Features by PLS', title_x=0.5, width=800)
fig.write_html(html_folder + 'PLS.html')
fig

## Ridge

In [3830]:
ridge = Ridge()
ridge_pipe = Pipeline([('preproc', preproc_pipe), ('ridge_model', ridge)])
parameters = {'ridge_model__alpha':np.logspace(-5, 3, 100)}
ridge_reg= GridSearchCV(ridge_pipe, parameters, scoring='neg_mean_squared_error',cv=5)
ridge_reg.fit(X_train, y_train)
print(mean_squared_error(y_test, ridge_reg.best_estimator_.predict(X_test), squared=False))
print(ridge_reg.best_estimator_.score(X_test, y_test))
print(ridge_reg.best_estimator_['ridge_model'].coef_)

0.3689514915443525
0.3821700544784029


## LASSO

In [3832]:
lasso = Lasso()
lasso_pipe = Pipeline([('preproc', preproc_pipe), ('lasso_model', lasso)])
parameters = {'lasso_model__alpha':np.logspace(-5, 3, 100)}
lasso_reg= GridSearchCV(lasso_pipe, parameters, scoring='neg_mean_squared_error',cv=5)
lasso_reg.fit(X_train, y_train)
print(mean_squared_error(y_test, lasso_reg.best_estimator_.predict(X_test), squared=False))
print(lasso_reg.best_estimator_.score(X_test, y_test))
print(lasso_reg.best_estimator_['lasso_model'].coef_)

0.3687951173437637
0.3826936582158955


## Elastic

In [151]:
elastic = ElasticNet()
elastic_pipe = Pipeline([('preproc', preproc_pipe), ('elastic_model', elastic)])
parameters = {'elastic_model__alpha':np.logspace(-6, 2, 10), 'elastic_model__l1_ratio':np.linspace(0.001,0.999,10)}
elastic_reg= GridSearchCV(elastic_pipe, parameters, scoring='neg_mean_squared_error',cv=5)
elastic_reg.fit(X_train, y_train)
print(mean_squared_error(y_test, elastic_reg.best_estimator_.predict(X_test), squared=False))
print(elastic_reg.best_estimator_.score(X_test, y_test))
print(elastic_reg.best_estimator_['elastic_model'].coef_)
print(elastic_reg.best_estimator_['elastic_model'].get_params)

0.3602652778688444
0.4020736203175904
